# Workflow

In [ ]:
import mne
import numpy as np 
import matplotlib.pyplot as plt
import torch
import os
from sklearn.pipeline import Pipeline

## Downloading the Data

To download the data, we'll use the Data folder which contains ways to download data. Here we'll start off by using the EEG_BCI motor imagery dataset.

If we want to download the entire dataset, we can do the following:

In [ ]:
from Data import download_EEGBCI 

In [ ]:
if input("Are you sure you want to download the entire dataset? (Y/N)") in "Yy":
    total_subjects = list(range(1, 110))
    runs = list(range(1, 15))
    download_EEGBCI(total_subjects, runs, './EEGData', False)
    print("Downloaded everything!")

else:
    print("Download cancelled.")

For now, we only want to download the first 3 subjects and the first 3 runs:

In [ ]:
subjects = [1, 2, 3]
runs = [1, 2, 3]
download_EEGBCI(subjects, runs, './EEGData', False)

## Loading the Data

This step is critical for actually getting the data into the ML algorithms. To do this, we'll have to rely on MNE's import functions to load the data into a format that we can use.

In [ ]:
main_folder = './EEGData/MNE-eegbci-data/files/eegmmidb/1.0.0'

subdirectories = [f.path for f in os.scandir(main_folder) if f.is_dir()]

opened_files = []
closed_files = []

for subdirectory in subdirectories:
    files = os.listdir(subdirectory)
    
    if len(files) > 0:
        for file in files:
            
            if file[-6:] == '01.edf':
                # This is data for eyes opened
                eyes_opened = os.path.join(subdirectory, file)
                print(eyes_opened)
                opened_files.append(eyes_opened)

            if file[-6:] == '02.edf':
                # This is data for eyes closed
                eyes_closed = os.path.join(subdirectory, file)
                print(eyes_closed)
                closed_files.append(eyes_closed)
    else:
        print(f"No files found in {subdirectory}")

In [ ]:
large_open_data = []
# This is a list of all the data for eyes opened

for data in opened_files:
    large_open_data.append(mne.io.read_raw_edf(data, preload=True, verbose=False).get_data(verbose=False))

large_closed_data = []
# This is a list of all the data for eyes closed

for data in closed_files:
    large_closed_data.append(mne.io.read_raw_edf(data, preload=True, verbose=False).get_data(verbose=False))


# This is the number of files for eyes opened
print(large_open_data.__len__())
# This is the number of files for eyes closed
print(large_closed_data.__len__())

Now, we've managed to go over how we load the data, let's do some basic data prep:

### More Detailed Ways of Working With MNE

In [ ]:
data_loc = "./EEGData/MNE-eegbci-data/files/eegmmidb/1.0.0/"
raw = mne.io.read_raw_edf('./EEGData/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R06.edf', preload=True)
annotations = raw.annotations
annotations 

In [ ]:
event_descriptions, event_counts = np.unique(annotations.description, return_counts=True)

# Print event descriptions and their counts
for desc, count in zip(event_descriptions, event_counts):
    print(f"Event: {desc}, Count: {count}")

# Select specific event type (replace 'event_type' with your desired event description)
event_type = 'event_type'
events = annotations[annotations.description == event_type]

# Print out event onset times and durations
for idx, event in enumerate(events):
    print(f"Event {idx + 1}: Onset: {event['onset']:.2f}, Duration: {event['duration']:.2f}")

# Replace 'your_file.edf' with the actual path to your EDF file
raw =  mne.io.read_raw_edf('./EEGData/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R06.edf', preload=True)

# Get EEG data for all channels
eeg_data = raw.get_data()

# Print the shape of the EEG data array
print("EEG data shape:", eeg_data.shape)

# List of EDF file paths
edf_files = ['./EEGData/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R06.edf', 
'./EEGData/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R10.edf',
'./EEGData/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R14.edf']

# List comprehension to read all EDF files
raw_list = [mne.io.read_raw_edf(edf_file, preload=True) for edf_file in edf_files]

# Extract EEG data for all channels from each raw object
eeg_data_list = [raw.get_data() for raw in raw_list]

# Print the shape of the EEG data arrays for each file
for idx, eeg_data in enumerate(eeg_data_list):
    print(f"EEG data shape for file {edf_files[idx]}:", eeg_data.shape)

raw_list[0].plot(scalings='auto', title='EEG Channels')

## Visualizing the Data

In [ ]:
def basic_plot(file_path):
    data = mne.io.read_raw_edf(file_path, preload=True, verbose=False)
    eeg_data = data.get_data()
    time = data.times
    plt.figure(figsize=(10, 6))
    for ch_idx in range(eeg_data.shape[0]):
        plt.plot(time, eeg_data[ch_idx], label=f'Channel {ch_idx + 1}')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.title('EEG Data for All Channels')
    return plt.show()

In [ ]:
def double_plot(file1, file2):
    fig, (ax1, ax2) = plt.subplots(2, 1)
    fig.suptitle('Eyes Opened vs Eyes Closed')

    data1 = mne.io.read_raw_edf(file1, preload=True, verbose=False)
    data1.filter(1.0, 30.0, fir_design="firwin", skip_by_annotation="edge", verbose=False)
    ica = mne.preprocessing.ICA(n_components=20, random_state=97, max_iter=800, verbose=False)
    ica.fit(data1, verbose=False)
    ica.exclude = [1, 2]
    data1 = ica.apply(data1, verbose=False)

    
    data2 = mne.io.read_raw_edf(file2, preload=True, verbose=False)
    data2.filter(1.0, 30.0, fir_design="firwin", skip_by_annotation="edge", verbose=False)
    ica = mne.preprocessing.ICA(n_components=20, random_state=97, max_iter=800, verbose=False)
    ica.fit(data2, verbose=False)
    ica.exclude = [1, 2]
    data2 = ica.apply(data2, verbose=False)
    

    good_channels1 = data1.info['ch_names']
    good_channels2 = data2.info['ch_names']

    eeg_data1 = data1.get_data(picks=good_channels1)
    eeg_data2 = data2.get_data(picks=good_channels2)

    time = data1.times

    ax1.plot(time, eeg_data1[0])
    ax1.set_title('Eyes Opened')
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Amplitude')
    for ch_idx in range(eeg_data1.shape[0]):
        ax1.plot(time, eeg_data1[ch_idx], label=f'Channel {ch_idx + 1}')
    ax1.set_xlim(0, 5)
    ax1.set_ylim(-0.0002, 0.0002)

    ax2.plot(time, eeg_data2[0])
    ax2.set_title('Eyes Closed')
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Amplitude')
    for ch_idx in range(eeg_data2.shape[0]):
        ax2.plot(time, eeg_data2[ch_idx], label=f'Channel {ch_idx + 1}')
    ax2.set_xlim(0, 5)
    ax2.set_ylim(-0.0002, 0.0002)
    return plt.show()

In [ ]:
basic_plot(opened_files[0])

In [ ]:
for i in range(1):
    double_plot(opened_files[i], closed_files[i])

In [ ]:
raw.compute_psd(fmax=50).plot(picks="data", exclude="bads")
raw.plot(duration=5, n_channels=30)

In [ ]:
# Load the raw EEG data
edf_file = './EEGData/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R06.edf'  # Replace with the path to your EDF file
raw = mne.io.read_raw_edf(edf_file, preload=True, verbose=False)


def compute_psd(data, fs, nperseg=256, noverlap=None):
    """
    Compute Power Spectral Density (PSD) using the Welch method.

    Parameters:
        data (array): EEG data array with shape (n_channels, n_samples).
        fs (float): Sampling frequency.
        nperseg (int): Length of each segment for PSD estimation.
        noverlap (int): Number of overlapping samples between segments.
    
    Returns:
        freqs (array): Frequency values.
        psd (array): Power Spectral Density values.
    """
    n_channels, n_samples = data.shape
    psd = np.zeros((n_channels, nperseg // 2 + 1))

    for ch_idx in range(n_channels):
        f, Pxx = plt.psd(data[ch_idx], Fs=fs, NFFT=nperseg, noverlap=noverlap)
        # Add a small epsilon to avoid zero values
        psd[ch_idx] = Pxx + 1e-10

    return f, psd


# Get the EEG data using .get_data()
eeg_data = raw.get_data()

# Set the sampling frequency
fs = raw.info['sfreq']

# Compute PSD using the custom function
freqs, psd = compute_psd(eeg_data, fs)

# Plot the PSD
plt.figure(figsize=(10, 6))  # Add this line to create a single figure
for ch_idx in range(eeg_data.shape[0]):
    plt.semilogy(freqs, psd[ch_idx], label=f'Channel {ch_idx + 1}')

plt.xlabel('Frequency (Hz)')
plt.ylabel('Power/Frequency (dB/Hz)')
plt.show()

## Preprocessing the Data